In [1]:
import numpy as np
from typing import List

from tqdm import tqdm

from data_loader import load_data
from schemes import *
from lin_algebra import *
from creator import Creator
import math
import time
import torch
from keras.layers import ZeroPadding2D
import low_lat
from logger import debug, debug_colours

from model_builder import *

### Low latency models

In [2]:
def mnist_fast(examples, mode='cifar'):
    if mode == 'normal':
        #scheme = get_ckks_scheme(512 * 16 * 2 * 1, primes=[60, 50, 50, 50, 50, 50, 60], scale_factor=50)
        scheme = get_ckks_scheme(512 * 16 * 1 * 1, primes=[32, 28, 28, 28, 28, 28, 32], scale_factor=28)

        creator = Creator(scheme)

        s = 2
        k = 5

        model = Model(creator)
        model.add(InputLayer(input_shape=(1, 30, 30)))
        model.add(ConvLayer(kernel_size=(k, k), stride=s, padding=0, num_maps=5, dense_mode=False))
        model.add(ActivationLayer(mode='square'))
        model.add(Flatten(groups=1))
        model.add(DenseLayer(output_length=100))
        model.add(ActivationLayer(mode='square'))
        model.add(DenseLayer(output_length=10))

        weights, test_features, preds_base, outputs_base = load_data(name='MNIST-CRYPTONETS')
        test_features = ZeroPadding2D()(test_features).numpy()

        model.compile(data_mode=1)
        model.load_weights(weights, scale=16)
        model.summary()
    elif mode == 'opt':
        scheme = get_ckks_scheme(512 * 16 * 1 * 1, primes=[32, 28, 28, 28, 28, 28, 32], scale_factor=28)
        creator = Creator(scheme)

        s = 1
        k = 3

        model = Model(creator)
        model.add(InputLayer(input_shape=(1, 30, 30)))
        model.add(ConvLayer(kernel_size=(k, k), stride=s, padding=0, num_maps=5, dense_mode=False))
        model.add(ActivationLayer(mode='square'))
        model.add(Flatten())
        model.add(DenseLayer(output_length=32))
        model.add(ActivationLayer(mode='square'))
        model.add(DenseLayer(output_length=10))

        weights, test_features, preds_base, outputs_base = load_data(name='MNIST-OPT')
        test_features = ZeroPadding2D()(test_features).numpy()

        model.compile(data_mode=1)
        model.load_weights(weights, scale=16)
        model.summary()
    elif mode == 'cifar':
        scheme = get_ckks_scheme(512 * 16 * 2 * 1, primes=[60, 30, 30, 30, 30, 30, 30, 30, 30, 60], scale_factor=30)
        #scheme = get_ckks_scheme(512 * 16 * 2 * 2, primes=[60, 50, 50, 50, 50, 50, 50, 50, 50, 60], scale_factor=50)
        creator = Creator(scheme)

        s = 1
        k = 3

        model = Model(creator)
        model.add(InputLayer(input_shape=(3, 32, 32)))
        model.add(ConvLayer(kernel_size=(k, k), stride=s, padding=0, num_maps=32, dense_mode=False))
        model.add(ActivationLayer(mode='square'))
        model.add(Flatten(groups=4))
        model.add(DenseLayer(output_length=512))
        model.add(Unflatten(channels=8))
        model.add(ConvLayer(kernel_size=(1, 1), stride=1, padding=0, num_maps=64, dense_mode=False))
        model.add(ActivationLayer(mode='square'))
        model.add(Flatten(groups=1))
        model.add(DenseLayer(output_length=256))
        model.add(ActivationLayer(mode='square'))
        model.add(DenseLayer(output_length=10))

        weights, test_features, preds_base, outputs_base = load_data(name='CIFAR_SVD')

        model.compile(data_mode=1)
        model.load_weights(weights, scale=16)
        model.summary()

    for i in range(examples):
        image = test_features[i]
        preds_base_i = preds_base[i]
        outputs_base_i = outputs_base[i]

        data = creator.encrypt_dense(mat=image)
        img_groups = creator.obtain_image_groups(mat=image, k=k, s=s)

        t1 = time.process_time()
        outputs = model.predict(input_data=img_groups,
                                     creator=creator,
                                     classes=10)

        output_probs = torch.softmax(torch.tensor(np.array(outputs, dtype=float)), 0).detach().numpy()

        debug('mnist_fast', str(time.process_time() - t1), debug_colours.PURPLE)
        debug('mnist_fast', str(np.argmax(outputs)) + ' ' + str(preds_base_i), debug_colours.PURPLE)

        if np.argmax(outputs) != preds_base_i:
            debug('mnist_fast', 'incorrect prediction!', debug_colours.PURPLE)

In [3]:
mnist_fast(examples=1, mode='cifar')

KeyboardInterrupt: 

In [3]:
mnist_fast(examples=1, mode='normal')

-------Summary of model-------
Input Layer, input shape = (1, 30, 30)
Conv2D layer, input shape = (1, 30, 30), output shape = (5, 13, 13), weights shape = (5, 5, 1, 5)
Activation layer, mode = square, input shape = (5, 13, 13), output shape = (5, 13, 13)
Flatten layer, input shape = (5, 13, 13), output shape = (1, 1, 845)
Dense layer, input shape = (1, 1, 845), output shape = (1, 1, 100), weights shape = (845, 100)
Activation layer, mode = square, input shape = (1, 1, 100), output shape = (1, 1, 100)
Dense layer, input shape = (1, 1, 100), output shape = (1, 1, 10), weights shape = (100, 10)



 20%|██        | 1/5 [00:00<00:00,  6.44it/s]

Layer 0::7.09999999948252e-06


  0%|          | 0/100 [00:00<?, ?it/s]

conv2d::#multCC: 125, #addPC: 5, #addCC: 0
Layer 1::0.7978030120000001
square::::#mult-CC: 5
Layer 2::0.05705209599999961
flatten::::#rots: 4, #add-CC: 4
Layer 3::0.0750121840000002


100%|██████████| 10/10 [00:00<00:00, 142.28it/s]


dense::#rots: 104, #multPC: 100, #addCC: 103, #addPC: 1
Layer 4::2.0637870830000002
square::::#mult-CC: 1
Layer 5::0.006884731999999616
dense::#rots: 14, #multPC: 10, #addCC: 13, #addPC: 1
Layer 6::0.08608794500000005
mnist_fast::3.0889864190000003
mnist_fast::7 7


In [5]:
mnist_fast(examples=10, mode='opt')

  0%|          | 0/5 [00:00<?, ?it/s]

-------Summary of model-------
Input Layer, input shape = (1, 30, 30)
Conv2D layer, input shape = (1, 30, 30), output shape = (5, 28, 28), weights shape = (3, 3, 1, 5)
Activation layer, mode = square, input shape = (5, 28, 28), output shape = (5, 28, 28)
Flatten layer, input shape = (5, 28, 28), output shape = (1, 1, 3920)
Dense layer, input shape = (1, 1, 3920), output shape = (1, 1, 32), weights shape = (3920, 32)
Activation layer, mode = square, input shape = (1, 1, 32), output shape = (1, 1, 32)
Dense layer, input shape = (1, 1, 32), output shape = (1, 1, 10), weights shape = (32, 10)

Layer 0::4.483000001442861e-06


 31%|███▏      | 10/32 [00:00<00:00, 90.22it/s]

conv2d::#multCC: 45, #addPC: 5, #addCC: 0
Layer 1::0.25180053799999946
square::::#mult-CC: 5
Layer 2::0.044511374000000714
flatten::::#rots: 4, #add-CC: 4
Layer 3::0.05537612999999908


  0%|          | 0/5 [00:00<?, ?it/s]

...

dense::#rots: 39, #multPC: 32, #addCC: 38, #addPC: 1
Layer 4::0.4930136350000005
square::::#mult-CC: 1
Layer 5::0.004514385000000232
dense::#rots: 13, #multPC: 10, #addCC: 12, #addPC: 1
Layer 6::0.0685121389999992
mnist_fast::0.9189069769999989
mnist_fast::5 5
Layer 0::3.5019999984342576e-06


 31%|███▏      | 10/32 [00:00<00:00, 88.22it/s]

conv2d::#multCC: 45, #addPC: 5, #addCC: 0
Layer 1::0.25105205200000213
square::::#mult-CC: 5
Layer 2::0.04298339700000042
flatten::::#rots: 4, #add-CC: 4
Layer 3::0.05596937099999977


100%|██████████| 10/10 [00:00<00:00, 178.45it/s]

dense::#rots: 39, #multPC: 32, #addCC: 38, #addPC: 1
Layer 4::0.4960499829999989
square::::#mult-CC: 1
Layer 5::0.004587438999998028
dense::#rots: 13, #multPC: 10, #addCC: 12, #addPC: 1
Layer 6::0.06923310300000196
mnist_fast::0.9207233539999997
mnist_fast::9 9


### MNIST-SIMD

### Base model

In [8]:
N = 8192 * 2
scheme = get_ckks_scheme(N, primes=[60, 50, 50, 50, 50, 50, 60], scale_factor=50)
creator = Creator(scheme)

In [9]:
model = Model(creator)
model.add(InputLayer(input_shape=(1, 30, 30)))
model.add(ConvLayer(kernel_size=(5, 5), stride=2, padding=0, num_maps=5))
model.add(ActivationLayer(mode='square'))
model.add(Flatten(groups=1))
model.add(DenseLayer(output_length=100))
model.add(ActivationLayer(mode='square'))
model.add(DenseLayer(output_length=10))

weights, test_features, preds_base, outputs_base = load_data(name='MNIST-CRYPTONETS')

test_features = test_features[:N // 2]
preds_base = preds_base[:N // 2]
outputs_base = outputs_base[:N // 2]

test_features = ZeroPadding2D()(test_features).numpy()

model.compile(data_mode=0)
model.load_weights(weights, scale=16)
model.summary()

-------Summary of model-------
Input Layer, input shape = (1, 30, 30)
Conv2D layer, input shape = (1, 30, 30), output shape = (5, 13, 13), weights shape = (5, 5, 1, 5)
Activation layer, mode = square, input shape = (5, 13, 13), output shape = (5, 13, 13)
Flatten layer, input shape = (5, 13, 13), output shape = (1, 1, 845)
Dense layer, input shape = (1, 1, 845), output shape = (1, 1, 100), weights shape = (845, 100)
Activation layer, mode = square, input shape = (1, 1, 100), output shape = (1, 1, 100)
Dense layer, input shape = (1, 1, 100), output shape = (1, 1, 10), weights shape = (100, 10)



In [10]:
print('Encrypting %s items' % len(test_features))

outputs = model.predict(input_data=creator.encrypt_simd(mat=test_features),
                             creator=creator, classes=N // 2)

  1%|          | 5/900 [00:00<00:18, 49.44it/s]

Encrypting 8192 items


  0%|          | 0/845 [00:00<?, ?it/s]

Layer 0::1.5184000005774578e-05


100%|██████████| 845/845 [04:08<00:00,  3.40it/s]


...

Adding biases
Layer 4::670.115942925


  0%|          | 0/10 [00:00<?, ?it/s]

square::::#mult-CC: 100
Layer 5::0.9335621830000491
-----Applying dense layer-----


100%|██████████| 10/10 [00:05<00:00,  1.96it/s]

Adding biases
Layer 6::5.312347123999871


In [11]:
outputs.shape

(1, 10, 8192)

In [12]:
classes = np.argmax(outputs, axis=1)[0]
classes.shape

(8192,)

In [13]:
preds_base.shape

(8192,)

In [14]:
wrongs = []
for i in range(len(classes)):
    if classes[i] != preds_base[i]:
        print(str(classes[i]) + ' ' + str(preds_base[i]))
        wrongs.append(i)
wrongs

[]

In [15]:
accuracy(classes, preds_base)

1.0

In [16]:
np.save('/home/han/Desktop/part-ii-project/notebooks/predictions/mnist_simd_classes', classes)

In [17]:
np.save('/home/han/Desktop/part-ii-project/notebooks/predictions/mnist_simd_outputs', outputs)

### Pruned model

In [2]:
N = 8192 * 2
scheme = get_ckks_scheme(N, primes=[60, 50, 50, 50, 50, 50, 60], scale_factor=50)
creator = Creator(scheme)

In [3]:
model = Model(creator)
model.add(InputLayer(input_shape=(1, 30, 30)))
model.add(ConvLayer(kernel_size=(5, 5), stride=2, padding=0, num_maps=5))
model.add(ActivationLayer(mode='square'))
model.add(Flatten(groups=1))
model.add(DenseLayer(output_length=100))
model.add(ActivationLayer(mode='square'))
model.add(DenseLayer(output_length=10))

weights, test_features, preds_base, outputs_base = load_data(name='LOTTERY')

test_features = test_features[:N // 2]
preds_base = preds_base[:N // 2]
outputs_base = outputs_base[:N // 2]

test_features = ZeroPadding2D()(test_features).numpy()

model.compile(data_mode=0)
model.load_weights(weights, scale=16)
model.summary()

-------Summary of model-------
Input Layer, input shape = (1, 30, 30)
Conv2D layer, input shape = (1, 30, 30), output shape = (5, 13, 13), weights shape = (5, 5, 1, 5)
Activation layer, mode = square, input shape = (5, 13, 13), output shape = (5, 13, 13)
Flatten layer, input shape = (5, 13, 13), output shape = (1, 1, 845)
Dense layer, input shape = (1, 1, 845), output shape = (1, 1, 100), weights shape = (845, 100)
Activation layer, mode = square, input shape = (1, 1, 100), output shape = (1, 1, 100)
Dense layer, input shape = (1, 1, 100), output shape = (1, 1, 10), weights shape = (100, 10)



In [4]:
print('Encrypting %s items' % len(test_features))

outputs = model.predict(input_data=creator.encrypt_simd(mat=test_features),
                             creator=creator, classes=N // 2)

  1%|          | 6/900 [00:00<00:16, 55.09it/s]

Encrypting 8192 items


  0%|          | 0/845 [00:00<?, ?it/s]

Layer 0::1.2283999996753892e-05


100%|██████████| 845/845 [00:50<00:00, 16.79it/s]


...

multPC 8316
addCC 8116
addPC 1
Layer 4::65.80428701599999


  0%|          | 0/10 [00:00<?, ?it/s]

square::::#mult-CC: 100
Layer 5::0.9588502770000105
-----Applying dense layer-----


100%|██████████| 10/10 [00:01<00:00,  8.56it/s]

multPC 228
addCC 208
addPC 1
Layer 6::1.3762352440000143


In [6]:
classes = np.argmax(outputs, axis=1)[0]
accuracy(classes, preds_base)

0.0859375